<a href="https://colab.research.google.com/github/hamiltonvalerio/DS_CNES_DATASUS/blob/main/CNES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import glob
import time
import requests
from io import StringIO
import unicodedata
import re
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


# SELECIONANDO NO GOOGLE DRIVE A TABELA DO MÊS 12/2020 DE NOMES DE EQUIPAMENTOS (tbEquipamento202012.csv) PARA FAZER UM JOIN COM O DATAFRAME de EQUIPAMENTOS TRAZENDO A NOMENCLATURA (Gama Camara, PET/CT)

In [2]:
url_eq_auxiliar = 'https://drive.google.com/file/d/19Qikm47nw3cy7FL6QwZj_4XmVHzz4Xp6/view?usp=sharing'
path_eq_auxiliar = 'https://drive.google.com/uc?export=download&id='+url_eq_auxiliar.split('/')[-2]
df_eq = pd.read_csv(path_eq_auxiliar,sep=';')

filtro_eq = ['Gama Camara', 'PET/CT']

result_eq = df_eq[df_eq['DS_EQUIPAMENTO'].str.contains('|'.join(filtro_eq))]

result_eq

,CO_EQUIPAMENTO,CO_TIPO_EQUIPAMENTO,DS_EQUIPAMENTO
7,1,1,Gama Camara
66,18,1,PET/CT


# ARRAY DE ESTADOS PARA PESQUISA

In [3]:
array_estados = np.array(["AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RO", "RS", "RR", "SC", "SE", "SP", "TO"])
pd.options.display.max_columns = None

# FUNÇÃO PARA RETIRAR ACENTOS

In [4]:
def removerAcentosECaracteresEspeciais(palavra):

    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', palavra)
    palavraSemAcento = u"".join([c for c in nfkd if not unicodedata.combining(c)])

    # Usa expressão regular para retornar a palavra apenas com números, letras e espaço
    return re.sub('[^a-zA-Z0-9 \\\]', '', palavraSemAcento)

    #crédito: https://gist.github.com/boniattirodrigo/67429ada53b7337d2e79

# MONTA O ARRAY DE LINKS DE EQUIPAMENTOS
Todos os arquivos do ano de 2020, mês 12, todos os Estados Brasileiros.

In [5]:
path_eq = 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQ'
array_urls_eq = []
campos_eq = ['CNES','CPF_CNPJ','CODEQUIP']

for est in array_estados:
    array_urls_eq.append(path_eq+""+est+"_2020.csv")
    
print(array_urls_eq)

['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAM_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAP_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQBA_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQCE_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQDF_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQES_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQGO_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQMA_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQMT_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQMS_20

# GERA O DATAFRAME DE EQUIPAMENTOS

In [6]:
timeStartEQ = time.time()
dfs_EQ = [pd.read_csv(url,sep=',', error_bad_lines=False, index_col=False, dtype='unicode', usecols = campos_eq) for url in array_urls_eq]
total_df_EQ = pd.concat(dfs_EQ, ignore_index=True)
tempoDeExecucaoEQ = (time.time() - timeStartEQ)
print('Tempo de execução em minutos: ' + str(tempoDeExecucaoEQ/60))

Tempo de execução em minutos: 1.608044425646464


In [7]:
total_df_EQ['CODEQUIP'] = total_df_EQ['CODEQUIP'].astype(int)

In [8]:
total_df_EQ.head(10)

,CNES,CPF_CNPJ,CODEQUIP
0,3006166,00000000000000,81
1,3006166,00000000000000,84
2,3006166,00000000000000,83
3,3006166,00000000000000,80
4,3382745,00000000000000,81
5,3382745,00000000000000,84
6,3382745,00000000000000,80
7,3382745,00000000000000,83
8,3393984,00000000000000,81
9,3393984,00000000000000,80


In [9]:
total_df_EQ.shape

(8596372, 3)

In [10]:
total_df_EQ.dtypes

CNES        object
CPF_CNPJ    object
CODEQUIP     int64
dtype: object

# FILTRA O DATAFRAME DE EQUIPAMENTOS COM OS FILTROS: coluna_codequip.values (1,18) - QUE CORRESPONDEM AOS CODIGOS DOS EQUIPAMENTOS: Gama Camara e PET/CT 

In [11]:
coluna_codequip = result_eq.loc[:,'CO_EQUIPAMENTO'].astype(int)
coluna_codequip.values

array([ 1, 18])

In [12]:
result_df_EQ = total_df_EQ.loc[total_df_EQ['CODEQUIP'].isin(coluna_codequip.values)]

In [13]:
result_df_EQ.shape

(7343, 3)

In [14]:
result_df_EQ = result_df_EQ.rename(columns={'CODEQUIP': 'CODEQUIP_TAB_EQ'})

In [15]:
result_df_EQ.head(10)

,CNES,CPF_CNPJ,CODEQUIP_TAB_EQ
921,2001977,00624104000177,1
1396,6225012,07616992000125,1
1538,6655653,11467937000198,1
2138,9467610,28685425000109,1
3609,2001977,00624104000177,1
4068,6225012,07616992000125,1
4210,6655653,11467937000198,1
4819,9467610,28685425000109,1
6299,2001977,00624104000177,1
6770,6225012,07616992000125,1


In [ ]:
result_df_EQ.to_csv('/drive/My Drive/DS_CNES/result_df_EQ.csv')

# MONTA O ARRAY DE LINKS ESTABELECIMENTOS

In [16]:
path_st = 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/ST'
array_urls_st = []
campos_st = ['CNES','CPF_CNPJ','ALVARA']

for est in array_estados:
    array_urls_st.append(path_st+""+est+"_2020.csv")
    
print(array_urls_st)

['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAM_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAP_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STBA_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STCE_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STDF_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STES_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STGO_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STMA_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STMT_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STMS_20

# GERA O DATAFRAME DE ESTABELECIMENTOS

In [17]:
timeStartST = time.time()
dfs_ST = [pd.read_csv(urlst, sep=',', error_bad_lines=False, index_col=False, dtype='unicode', usecols = campos_st) for urlst in array_urls_st]
total_df_ST = pd.concat(dfs_ST, ignore_index=True)
tempoDeExecucaoST = (time.time() - timeStartST)
print('Tempo de execução em minutos: ' + str(tempoDeExecucaoST/60))

Tempo de execução em minutos: 2.2570220947265627


In [18]:
total_df_ST.shape

(3581555, 3)

In [19]:
total_df_ST.head(10)

,CNES,CPF_CNPJ,ALVARA
0,3006166,00000000000000,NaN
1,3382745,00000000000000,NaN
2,3393984,00000000000000,NaN
3,3638685,00000000000000,NaN
4,5403669,00000000000000,NaN
5,5701929,00000000000000,NaN
6,6514669,00000000000000,NaN
7,7026641,00000000000000,NaN
8,7245890,10923709000201,239/2013
9,7669135,17390533000120,NaN


# FAZ O MERGE (RIGHT JOIN NO SQL) USANDO COMO CHAVE O VALOR CNES E REMOVENDO OS DUPLICADOS

Aqui tenho um resultado de todas as empresas da base CNES que trabalham (ou tenham!?) os equipamentos GAMA CAMARA e PET/CT. Nesta lista podemos conferir se, pelo CNES, temos o Alvará.
Com base nesta lista é possível importar a base de Clientes da Cnen e fazer uma comparação utilizando o CNES ou CNPJ cadastrado. 

In [20]:
df_lista_cnes = pd.merge(total_df_ST, result_df_EQ, on="CNES", how="left").drop_duplicates()
df_lista_cnes = (df_lista_cnes.rename(columns={'CPF_CNPJ_x': 'CNPJ'})).drop(columns=['CPF_CNPJ_y'])
df_lista_cnes['CNPJ'] = df_lista_cnes['CNPJ'].str.replace(".","")
df_lista_cnes['CNPJ'] = df_lista_cnes['CNPJ'].str.replace("/","")
df_lista_cnes['CNPJ'] = df_lista_cnes['CNPJ'].str.replace("-","")
df_lista_cnes['CNPJ'] = df_lista_cnes['CNPJ'].str.replace(" ","")


In [58]:
filtro_cnpj_lista = ['12343023000188']
result_cnpj_lista = df_lista_cnes[df_lista_cnes['CNPJ'].str.contains('|'.join(filtro_cnpj_lista))]

result_cnpj_lista

,CNES,CNPJ,ALVARA,CODEQUIP_TAB_EQ
2834219,6808050,12343023000188,35162000486400008516,1.0


In [22]:
df_lista_cnes.shape

(375854, 4)

In [48]:
df_lista_cnes = df_lista_cnes.drop_duplicates(subset=['CNPJ'])

In [49]:
df_lista_cnes.shape

(270227, 4)

In [23]:

df_lista_cnes.head(10)

,CNES,CNPJ,ALVARA,CODEQUIP_TAB_EQ
0,3006166,00000000000000,NaN,NaN
1,3382745,00000000000000,NaN,NaN
2,3393984,00000000000000,NaN,NaN
3,3638685,00000000000000,NaN,NaN
4,5403669,00000000000000,NaN,NaN
5,5701929,00000000000000,NaN,NaN
6,6514669,00000000000000,NaN,NaN
7,7026641,00000000000000,NaN,NaN
8,7245890,10923709000201,239/2013,NaN
9,7669135,17390533000120,NaN,NaN


In [ ]:
df_lista_cnes.to_csv('/drive/My Drive/DS_CNES/df_lista_cnes_leftjoin.csv')

In [60]:
df_lista_cnes.to_excel('/drive/My Drive/DS_CNES/df_lista_cnes.xlsx',sheet_name='df_lista_cnes')

# CARREGA DO GOOGLE DRIVE A LISTA DE CLIENTES DISPONIBILIZADA PELA CNEN
Ao mesmo tempo que carrega a lista, que esta em excel, é formatado o campo de CNPJ para Join com o resultado da base CNES acima.

In [24]:
url_lista_clientes_cnen = 'https://drive.google.com/file/d/1OvY1NsR0Bn8eDq2I28TRvwjQPEdPywAh/view?usp=sharing'
path_lista_clientes_cnen = 'https://drive.google.com/uc?export=download&id='+url_lista_clientes_cnen.split('/')[-2]
df_lista_clientes_cnen = pd.read_excel(path_lista_clientes_cnen, index_col=0).drop(columns=['seq'])
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace(".","")
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace("/","")
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace("-","")
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace(" ","")
df_lista_clientes_cnen['RAZAOSOCIAL'] = df_lista_clientes_cnen['RAZAOSOCIAL'].apply(lambda x:removerAcentosECaracteresEspeciais(x))
df_lista_clientes_cnen['UNIDADE'] = df_lista_clientes_cnen['UNIDADE'].apply(lambda x:removerAcentosECaracteresEspeciais(x))
df_lista_clientes_cnen['RESPONSAVEL'] = df_lista_clientes_cnen['RESPONSAVEL'].apply(lambda x:removerAcentosECaracteresEspeciais(x))
df_lista_clientes_cnen['ENDERECO'] = df_lista_clientes_cnen['ENDERECO'].apply(lambda x:removerAcentosECaracteresEspeciais(x))
df_lista_clientes_cnen['MUNICIPIO'] = df_lista_clientes_cnen['MUNICIPIO'].apply(lambda x:removerAcentosECaracteresEspeciais(x))




In [25]:
df_lista_clientes_cnen.head(5)

,CODIGORESP,CNPJ,RAZAOSOCIAL,UNIDADE,RESPONSAVEL,MATRICULACNEN,ENDERECO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
CODIGOCLIENTE,,,,,,,,,,,,,
5002184,1,21554423000123,INSTITUTO ONCOLOGICO LTDA JUIZ DE FORAMG,MEDICINA NUCLEAR,GUSTAVO SFFEIR C DE MAGALHAES,16404,RUA SANTOS DUMONT,56,NaN,GRANBERY,JUIZ DE FORA,MG,36010510
5002672,1,33205964000125,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141
5009782,1,15166416000151,REAL SOCIEDADE PORTUGUESA DE BENEFICIENCIA 16 ...,MEDICINA NUCLEAR,EDMARIO ANTONIO G COSTA,11672,AV PRINCESA ISABEL,914,NaN,BARRA AVENIDA,SALVADOR,BA,40140901
5016363,1,75033308000132,CENTRO DE MEDICINA NUCLEAR DO PARANA LTDA SC,MEDICINA NUCLEAR,TAIZA PARENTE ZUKOVSKI,15390,RUA PDE ILDEFONSO,158,NaN,BATEL,CURITIBA,PR,80240160
5017408,1,33804212000180,CLINICA SAO CARLOS SA,SERV MED NUCLEAR,CARLOS AFFONSO FUSER,11663,RUA HUMAITA,296,NaN,BOTAFOGO,RIO DE JANEIRO,RJ,22261001


In [ ]:
df_lista_clientes_cnen.to_csv('/drive/My Drive/DS_CNES/df_lista_clientes_cnen.csv')

In [ ]:
df_lista_clientes_cnen.to_excel('/drive/My Drive/DS_CNES/df_lista_clientes_cnen.xlsx')

# FAZ O MERGE DA LISTA DE CLIENTES COM A LISTA DA BASE CNES
Utiliza como chave o campo CNPJ

In [61]:
print("Quantidade de linhas tabela Clientes/CNEN: "+str(df_lista_clientes_cnen.shape)+
      "\nQuantidade de linhas tabela CNES/DATASUS: "+str(df_lista_cnes.shape))


Quantidade de linhas tabela Clientes/CNEN: (443, 13)
Quantidade de linhas tabela CNES/DATASUS: (270227, 4)


In [62]:
df_resultado_final = pd.merge(df_lista_clientes_cnen, df_lista_cnes, left_on="CNPJ", right_on="CNPJ", how="inner")
df_resultado_final['ALVARA'] = df_resultado_final['ALVARA'].str.replace(",","")
df_resultado_final['COMPLEMENTO'] = df_resultado_final['COMPLEMENTO'].str.replace(",","")

In [63]:
print("Quantidade de linhas do resultado final: "+str(df_resultado_final.shape))


Quantidade de linhas do resultado final: (392, 16)


In [64]:
df_resultado_final.to_excel('/drive/My Drive/DS_CNES/df_resultado_final_inner.xlsx')

#Condição CNEN !-> CNES
Abaixo aplico a seguinte condição: Trazer todos os clientes da tabela da CNEN que não estão inseridos na tabela da CNES.

In [68]:
df_resultado_final_inverso = df_lista_clientes_cnen[~df_lista_clientes_cnen['CNPJ'].isin(df_lista_cnes['CNPJ'])]

In [70]:
df_resultado_final_inverso.shape

(51, 13)

In [72]:
df_resultado_final_inverso.to_excel('/drive/My Drive/DS_CNES/df_resultado_final_inverso.xlsx')

In [52]:
pd.merge(df_lista_clientes_cnen, df_lista_cnes,on='CNPJ',how='outer',indicator=True)

,CODIGORESP,CNPJ,RAZAOSOCIAL,UNIDADE,RESPONSAVEL,MATRICULACNEN,ENDERECO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP,CNES,ALVARA,CODEQUIP_TAB_EQ,_merge
0,1.0,21554423000123,INSTITUTO ONCOLOGICO LTDA JUIZ DE FORAMG,MEDICINA NUCLEAR,GUSTAVO SFFEIR C DE MAGALHAES,16404.0,RUA SANTOS DUMONT,56,NaN,GRANBERY,JUIZ DE FORA,MG,36010510,2153106,398/2005,1.0,both
1,1.0,33205964000125,CLINICA DE MEDICINA NUCL VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SA VILELA PEDRAS,11595.0,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141,3178447,09/97/027417/2019,18.0,both
2,1.0,15166416000151,REAL SOCIEDADE PORTUGUESA DE BENEFICIENCIA 16 ...,MEDICINA NUCLEAR,EDMARIO ANTONIO G COSTA,11672.0,AV PRINCESA ISABEL,914,NaN,BARRA AVENIDA,SALVADOR,BA,40140901,0004251,PROT-0311190010223,1.0,both
3,1.0,75033308000132,CENTRO DE MEDICINA NUCLEAR DO PARANA LTDA SC,MEDICINA NUCLEAR,TAIZA PARENTE ZUKOVSKI,15390.0,RUA PDE ILDEFONSO,158,NaN,BATEL,CURITIBA,PR,80240160,0015865,2019000000003,1.0,both
4,1.0,33804212000180,CLINICA SAO CARLOS SA,SERV MED NUCLEAR,CARLOS AFFONSO FUSER,11663.0,RUA HUMAITA,296,NaN,BOTAFOGO,RIO DE JANEIRO,RJ,22261001,3009947,E-08/101.518/1997,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270277,NaN,00005094299123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0440361,2001179252,NaN,right_only
270278,NaN,27797198000132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0413097,405/2020,NaN,right_only
270279,NaN,37130185000169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0426059,20200117288,NaN,right_only
270280,NaN,36138785000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0426180,2020013929,NaN,right_only


In [53]:
df_resultado_final_join.shape

(559, 15)

In [ ]:
df_resultado_final.to_csv('/drive/My Drive/DS_CNES/df_resultado_final_left.csv')


In [ ]:
df_resultado_final.to_excel('/drive/My Drive/DS_CNES/df_resultado_final_left.xlsx')